In [1]:
import os
from sklearn.preprocessing import StandardScaler
#import wfdb as wf
import numpy as np
import matplotlib.pyplot as plt
from biosppy.signals import ecg
import glob
from scipy import signal
import pandas as pd
from tensorflow.python.keras.layers import Dense, Convolution1D, Convolution2D,MaxPool1D, Flatten, Dropout
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers.normalization import BatchNormalization
import tensorflow.python.keras
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.utils.np_utils import to_categorical
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter
from sklearn import preprocessing
import pandas as pd
import numpy as np
import os,glob
import math
import fnmatch
import re
import gc
import shutil
from joblib import Parallel, delayed
#import matplotlib.pyplot as plt
import scipy.signal as scipysi
import xgboost as xgb
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.feature_selection import SelectFromModel
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional, GRU
import shutil
#import matplotlib as plt
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape,CuDNNLSTM,GlobalMaxPooling1D, MaxPool2D,Flatten
from tensorflow.python.keras.layers import Conv1D, Conv2D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout, GlobalAveragePooling2D,Concatenate
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from imblearn.over_sampling import SMOTE, ADASYN, SVMSMOTE
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras import initializers
from tensorflow.keras import regularizers, constraints
import tensorflow.keras.backend as K


from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap

from scipy import signal
from scipy.fft import fftshift

/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
#constants

TARGET_SAMPLING_RATE=250
DATA_SAMPLING_RATE=500
INPUT_BEAT_SIZE=300

#paths
root_path='/Users/aring/Desktop/ECG-identification/Arnold-data'

DataFilePath = root_path+'/NoiseReductionData/'
LeadInfoFilePath = root_path+'/DiagnosisFiles/MissingLeadInfo/'
RepDiagnosisFileName = root_path+'/DiagnosisFiles/RepatTranslationDiagAfterRemDupAndBDQ_CHS20200531_ToArinG.xlsx'
FullDiagnosisFileName = root_path+'/DiagnosisFiles/TranslationDiagAfterRemDupAndBDQ_CHS20200531_ToArinG.xlsx'

In [3]:
FullDiagnosisDF = pd.read_excel(FullDiagnosisFileName)
print(FullDiagnosisDF.shape)
AllDiagnosisDF['PatientID'] = AllDiagnosisDF['PatientID'].astype('str')
AllDiagnosisDF['PatientID_new'] = le.fit_transform(AllDiagnosisDF['PatientID'])


(45308, 7)


In [4]:
AllDiagnosisDF.head()

,FileName,PatientID,DOB,Age,Gen,TimeAcquisition,Translation,PatientID_new
0,MUSE_20180113_180130_23000,219,1011931.0,85,Male,29/12/2016 10:52:16,['Sinus_Tachycardia'],1673
1,MUSE_20180113_180131_30000,219,1011931.0,86,Male,29/05/2017 10:10:54,['Sinus_Tachycardia'],1673
2,MUSE_20180113_180139_35000,219,1011931.0,86,Male,03/06/2017 07:34:27,['Sinus_Tachycardia'],1673
3,MUSE_20180113_180143_59000,219,1011931.0,86,Male,04/11/2017 11:46:50,['Sinus_Tachycardia'],1673
4,MUSE_20180113_131105_22000,542,1011931.0,86,Female,26/01/2017 12:50:33,[ 'Ventricular_Escape_Beat'],2269


In [12]:
AllDiagnosisDF['FileName'] = AllDiagnosisDF.apply(lambda x: x['FileName']+'.csv',axis=1)


In [13]:
AllDiagnosisDF.head()

,FileName,PatientID,DOB,Age,Gen,TimeAcquisition,Translation,PatientID_new
0,MUSE_20180113_180130_23000.csv,219,1011931.0,85,Male,29/12/2016 10:52:16,['Sinus_Tachycardia'],1673
1,MUSE_20180113_180131_30000.csv,219,1011931.0,86,Male,29/05/2017 10:10:54,['Sinus_Tachycardia'],1673
2,MUSE_20180113_180139_35000.csv,219,1011931.0,86,Male,03/06/2017 07:34:27,['Sinus_Tachycardia'],1673
3,MUSE_20180113_180143_59000.csv,219,1011931.0,86,Male,04/11/2017 11:46:50,['Sinus_Tachycardia'],1673
4,MUSE_20180113_131105_22000.csv,542,1011931.0,86,Female,26/01/2017 12:50:33,[ 'Ventricular_Escape_Beat'],2269


In [14]:
AllDiagnosisDF[['FileName']].to_csv('/Users/aring/Desktop/ECG-identification/filenames.csv', index=False)

In [15]:
AllDiagnosisDF.shape

(45309, 8)